In [60]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import math

In [56]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome()

In [61]:
stealth(driver,
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
)

In [62]:
driver.get("https://animepahe.ru")

# input = driver.find_element(By.CLASS_NAME, "input-search")
# input.send_keys("One piece")

In [63]:
# Get search results
# search_result_wrap = driver.find_element(By.CLASS_NAME, "search-results-wrap")

# while True:
#     try:
#         ul =  search_result_wrap.find_element(By.CLASS_NAME, "search-results")
#         li = ul.find_elements(By.TAG_NAME, "li")
#         for anime in li:
#             a = anime.find_element(By.TAG_NAME, "a")
#             result_title = a.find_element(By.CLASS_NAME, "result-title")
#             print (result_title.text.strip())
#         break
#     except Exception as e:
#         print (f"Couldn't find the results. Retrying in 5s: {e}")
#         time.sleep(5)

In [64]:
import requests

response = requests.get("https://animepahe.ru/api?m=search&q=One piece")
jsonres = None

if response.status_code == 200:
    # Print the response content (usually in JSON format for APIs)
    jsonres = response.json()
    print(jsonres)
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code} - {response.text}")

{'total': 61, 'per_page': 8, 'current_page': 1, 'last_page': 8, 'from': 1, 'to': 8, 'data': [{'id': 4, 'title': 'One Piece', 'type': 'TV', 'episodes': 0, 'status': 'Currently Airing', 'season': 'Fall', 'year': 1999, 'score': 8.71, 'poster': 'https://i.animepahe.ru/posters/355e6e3127aa31f0d806114169b52c4fb6da4b87df7f9c1809b9e3de97b8aac5.jpg', 'session': '1f0c2e22-a432-cda9-7ac8-e6cbc7852a93'}, {'id': 3778, 'title': 'One Piece Movie 01', 'type': 'Movie', 'episodes': 1, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2000, 'score': 7.09, 'poster': 'https://i.animepahe.ru/posters/239bba86be7f81f2f7bf011f997136f67cecfa0fcaaac4b0747cdadefb45e845.jpg', 'session': 'dcdc6e99-efcc-8a0a-505f-5d93ce7c0ace'}, {'id': 3838, 'title': 'One Piece Film: Gold - Cine Mike Popcorn Kokuchi', 'type': '', 'episodes': 1, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2016, 'score': 6.36, 'poster': 'https://i.animepahe.ru/posters/7eaa1bebf5c244a8a34e95c0d3eb45d3cc0801f55c8f0e194e8b9934e8e228a0

In [65]:
driver.get(f"https://animepahe.ru/anime/{jsonres['data'][0]['session']}")

In [66]:
response = requests.get(f"https://animepahe.ru/api?m=release&id={jsonres['data'][0]['session']}&sort=episode_asc&page=1")
anime_details = response.json()

anime_details

{'total': 1089,
 'per_page': 30,
 'current_page': 1,
 'last_page': 37,
 'next_page_url': 'https://animepahe.ru/api?page=2',
 'prev_page_url': None,
 'from': 1,
 'to': 30,
 'data': [{'id': 36600,
   'anime_id': 4,
   'episode': 1,
   'episode2': 0,
   'edition': '',
   'title': '',
   'snapshot': 'https://i.animepahe.ru/snapshots/c311c5b993085c3e9de8e44387a0cb48b69fe2c3ba4fcf77d52f1ab4f890c12a.jpg',
   'disc': '',
   'audio': 'jpn',
   'duration': '00:25:00',
   'session': '7dcd6ed708bd2cf222a21ec3be167142043692ee3c05a6bdd4a89a4cfeaebdf5',
   'filler': 0,
   'created_at': '2019-12-12 09:53:44'},
  {'id': 36601,
   'anime_id': 4,
   'episode': 2,
   'episode2': 0,
   'edition': '',
   'title': '',
   'snapshot': 'https://i.animepahe.ru/snapshots/ba684b711d8d1382d9934b69fac36dd8a55bcd1d81cd04ea668f1173fc503761.jpg',
   'disc': '',
   'audio': 'jpn',
   'duration': '00:25:00',
   'session': '538a18d996aa035b044cfd23ce6517d0ef226f3d1ef8eea2678a9a32c62bb6ff',
   'filler': 0,
   'created_at':

In [67]:
# getting the episodes
def get_episode_link(episode, total):
    page = math.ceil(episode/30)
    response = requests.get(f"https://animepahe.ru/api?m=release&id={jsonres['data'][0]['session']}&sort=episode_asc&page={page}").json()
    position = (episode - (page - 1) * 30) - 1
    return f"https://animepahe.ru/play/{jsonres['data'][0]['session']}/{response['data'][position]['session']}"
        
link = get_episode_link(60, anime_details['total'])
link

'https://animepahe.ru/play/1f0c2e22-a432-cda9-7ac8-e6cbc7852a93/a1f3c490883a2f81cd39f1c915b8f0487d1827687d15531263ac3f600f331a77'

In [53]:

requests.get(
    "https://animepahe.ru/play/1f0c2e22-a432-cda9-7ac8-e6cbc7852a93/a1f3c490883a2f81cd39f1c915b8f0487d1827687d15531263ac3f600f331a77",
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15'
    }
).text

'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><link href="/cdn-cgi/styles/challenges.css" rel="stylesheet"><meta http-equiv="refresh" content="375"></head><body class="no-js"><div class="main-wrapper" role="main"><div class="main-content"><noscript><div id="challenge-error-title"><div class="h2"><span class="icon-wrapper"><div class="heading-icon warning-icon"></div></span><span id="challenge-error-text">Enable JavaScript and cookies to continue</span></div></div></noscript></div></div><script>(function(){window._cf_chl_opt={cvId: \'2\',cZone: "animepahe.ru",cType: \'managed\',cNounce: \'90456\',cRay: \'841e86fb5b29d6de\',cHash: \'94b828b61a76399\',cUPMDTk: "\\/play\\/1f0c2e22-a432-cda9-7ac8-e6cbc7852a93\\/a1f3c490883a2f81cd39f1c9

In [68]:
driver.get(link)

In [69]:
# Get quality lists
qualities = []

for a in driver.find_elements(By.CSS_SELECTOR, "#pickDownload .dropdown-item"):
    quality = {
        "name": a.get_attribute("innerHTML").split(" ")[2],
        "link": a.get_attribute("href")
    }
    qualities.append(quality)

qualities

[]

In [13]:
qualities[0]['link']

'https://pahe.win/jLTYJ'

In [15]:
driver.get(qualities[0]['link'])

In [16]:
# Get the continue button
link_to_download = ""

while True:
    try:
        a = driver.find_element(By.XPATH, "//a[text()='Continue']")
        link_to_download = a.get_attribute("href")
        break
    except:
        print("Couldn't find button, waiting 7s")
        time.sleep(7)
 
link_to_download

Couldn't find button, waiting 7s


'https://kwik.cx/f/abIaGBnaQwUk'

In [17]:
driver.get(link_to_download)

In [18]:
# Find button and click it
link = driver.find_element(By.CSS_SELECTOR, ".main .download form").get_attribute("action")
token = driver.find_element(By.CSS_SELECTOR, ".main .download form input").get_attribute("value")
print("Link:", link)
print("Token:", token)


Link: https://kwik.cx/d/abIaGBnaQwUk
Token: uw6NEU7VCFKf540KHPZ08pelX51IcoB5UPqdEC7T


In [19]:
# cookies = {}
# for cookie in driver.get_cookies():
#     cookies[cookie['name']] = cookie['value']
# cookies

cookies = '; '.join([
    f"{cookie['name']}={cookie['value']}"
    for cookie in driver.get_cookies()
])
cookies

'kwik_session=eyJpdiI6Ik1wbkllY0p6OHhpODhYQlB2SU9mTWc9PSIsInZhbHVlIjoiOVpEdVRiYS8zbHJOVW0yZDVIRDFSajlrdWhjQmk2a3hwRFNFZGp2NGdLRVQvNEVNRmROT1FRRGRZZFBSUklBWkM1bGN6WEZ0Lzl3VTc5R09hRHJ6SEMyTFIzL2dwMUE1RXhJeEVEZ0ZoSFVxcnRKbkloeFNtRVRTdDZRdzUwMXYiLCJtYWMiOiI4NjYzNmY2ODljY2YyNjUyMGJkOGExMzBmZTk5ZWEwNTRjZDVlZWQwOWJjYjMwMjU1YjZmNjgxY2RhNjRjNmFkIiwidGFnIjoiIn0%3D; srv=s0; ppu_show_on_4e5e04716f26fd21bf611637f4fb8a46=1; sb_page_e1010ee4b61613b1b253d71d1c531c2e=1; sb_main_e1010ee4b61613b1b253d71d1c531c2e=1; sb_count_e1010ee4b61613b1b253d71d1c531c2e=1; sb_onpage_e1010ee4b61613b1b253d71d1c531c2e=1; ppu_main_4e5e04716f26fd21bf611637f4fb8a46=1; ppu_exp_4e5e04716f26fd21bf611637f4fb8a46=1704583776595; dom3ic8zudi28v8lr6fgphwffqoz0j6c=815d8304-d961-43eb-b897-09371d8d2f7f%3A1%3A1; pbpr0tpuw4isk85t8yg3jb2lj5vqf=sortveil.com'

In [20]:
data = {
    "_token": token
}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0',
  'Referer': 'https://kwik.cx/f/r6gCLSqIe9Nh',
  'Origin': 'https://kwik.cx',
  'Host': 'kwik.cx',
  'Cookie': cookies
}

In [21]:

data = {
    "_token": token
}
response = requests.post(link, data=data, headers=headers, stream=True)
file_name = response.headers.get('Content-Disposition').split('=')[1]
print(file_name)

response.raise_for_status()

total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kibibyte

progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

with open(file_name, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)

progress_bar.close()

AnimePahe_One_Piece_-_002_720p_HorribleSubs.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████| 125M/125M [00:58<00:00, 2.13MiB/s]
